In [ ]:
import numpy as np
import random

In [ ]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]

print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [ ]:
### Shuffle the cards randomly. Each player gets 8 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3")
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 1), (12, 1), (9, 3), (14, 1), (11, 3), (9, 2), (13, 2), (9, 1)]
[(9, 0), (13, 1), (12, 0), (11, 2), (14, 3), (10, 0), (14, 2), (10, 2)]


In [ ]:
class Player():
    def __init__(self, name):

        self.name = name
        self.cards = None

    ### -------------------------------------------------------------

    ### TO BE IMPLEMENTED - player's strategy
    ### input: declared card, i.e., the card which is supposed
    ### to be the top card of the pile: If None - you can put any card you want because
    ### (a) it is the first turn (pile is empty) or (b) some cards were drawn in the previous turn)
    ### output: - player's true decision, player's declaration (if not equal - (s)he cheats)

    def putCard(self, declared_card):
        ### DO NOT REMOVE TRUE CARD cards.remove!!!
        ### return an object (not id): self.cards[id], not id
        ### for instance: return self.cards[0], self.cards[0]
        ### IMPORTANT: If you want to draw cards instead of put, return "draw"
        ### for instance: return "draw"
        return self.cards[0], self.cards[0]

    ### TO BE IMPLEMENTED - Decide whether to check or not opponent's move (return True or False)
    def checkCard(self, opponent_declaration):
        pass

    ### Notification sent at the end of a round
    ### One may implement this method, capture data, and use it to get extra info
    ### -- checked = TRUE -> someone checked. If FALSE, the remaining inputs do not play any role
    ### -- iChecked = TRUE -> I decided to check my opponent (so it was my turn);
    ###               FALSE -> my opponent checked and it was his turn
    ### -- iDrewCards = TRUE -> I drew cards (so I checked but was wrong or my opponent checked and was right);
    ###                 FALSE -> otherwise
    ### -- revealedCard - some card (X, Y). Only if I checked.
    ### -- noTakenCards - number of taken cards
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " +
                      str(iDrewCards) + "; revealed card = " +
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))


    ### -------------------------------------------------------------

    ### Init player's hand
    def startGame(self, cards):
        self.cards = cards

    ### Add some cards to player's hand (if (s)he checked opponent's move, but (s)he was wrong)
    def takeCards(self, cards_to_take):
        self.cards = self.cards + cards_to_take

In [ ]:
# Some examplary random player

class RandomPlayer(Player):

    ### player's random strategy
    def putCard(self, declared_card):

        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card

        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])

        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)

        ### Yet, declared card should be no worse than a card on the top of the pile .
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration

    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])


In [ ]:
class Game():
    def __init__(self, players, log = True):
        self.players = players
        self.deck = getDeck()
        self.player_cards = getShuffled(self.deck)
        self.game_deck = self.player_cards[0] + self.player_cards[1]

        self.cheats = [0, 0]
        self.moves = [0, 0]
        self.checks = [0, 0]
        self.draw_decisions = [0, 0]

        for i, cards in zip([0, 1], self.player_cards):
            self.players[i].startGame(cards.copy())
            if log:
                print("Player (" + str(i + 1) + "): " + self.players[i].name + " received:")
                print(self.players[i].cards)

        ### Which card is on top
        self.true_card = None
        ### Which card was declared by active player
        self.declared_card = None

        ### Init pile: [-1] = top card
        self.pile = []

        ### Which player moves
        self.player_move = np.random.randint(2)

    def takeTurn(self, log = True):

        self.player_move = 1 - self.player_move

        if log:
            print("")
            print("")
            print("==== CURRENT STATE ================================")
            print("==== " + self.players[self.player_move].name + " MOVES ====")
            print("Player (0): " + self.players[0].name + " hand:")
            print(self.players[0].cards)
            print("Player (1): " + self.players[1].name + " hand:")
            print(self.players[1].cards)
            print("Pile: ")
            print(self.pile)
            print("Declared top card:")
            print(self.declared_card)
            print("")

        activePlayer = self.players[self.player_move]
        opponent = self.players[1 - self.player_move]
        self.moves[self.player_move] += 1

        self.previous_declaration = self.declared_card
        decision = activePlayer.putCard(self.declared_card)

        if decision == "draw":

            if log: print("[+] " + activePlayer.name + " decides to draw cards")

            self.draw_decisions[self.player_move] += 1

            toTake = self.pile[max([-3, -len(self.pile)]):]
            for c in toTake: self.pile.remove(c)
            activePlayer.takeCards(toTake)
            for c in toTake: self.player_cards[self.player_move].append(c)

            self.declared_card = None
            self.true_card = None

            activePlayer.getCheckFeedback(False, False, False, None, None, log)
            opponent.getCheckFeedback(False, False, False, None, None, log)

        else:
            self.true_card, self.declared_card = decision
            if self.true_card != self.declared_card: self.cheats[self.player_move] += 1

            if log: print("[+] " + activePlayer.name + " puts " + str(self.true_card) +
                          " and declares " + str(self.declared_card))

            if not self.debugMove(): return False, self.player_move

            activePlayer.cards.remove(self.true_card)
            self.player_cards[self.player_move].remove(self.true_card)
            self.pile.append(self.true_card)

            if opponent.checkCard(self.declared_card):

                self.checks[1 - self.player_move] += 1

                if log: print("[!] " + opponent.name + ": " + "I want to check")
                toTake = self.pile[max([-3, -len(self.pile)]):]
                for c in toTake: self.pile.remove(c)

                if not self.true_card == self.declared_card:
                    if log: print("\tYou are right!")
                    activePlayer.takeCards(toTake)

                    activePlayer.getCheckFeedback(True, False, True, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, False, tuple(toTake[-1]), len(toTake), log)

                    for c in toTake: self.player_cards[self.player_move].append(c)
                else:
                    if log: print("\tYou are wrong!")
                    opponent.takeCards(toTake)

                    activePlayer.getCheckFeedback(True, False, False, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, True, tuple(toTake[-1]), len(toTake), log)

                    for c in toTake: self.player_cards[1 - self.player_move].append(c)

                if log:
                    print("Cards taken: ")
                    print(toTake)

                self.declared_card = None
                self.true_card = None
            else:
                activePlayer.getCheckFeedback(False, False, False, None, None, log)
                opponent.getCheckFeedback(False, False, False, None, None, log)


        if not self.debugGeneral(): return False, self.player_move
        return True, self.player_move

    def isFinished(self, log = True):
        if len(self.players[self.player_move].cards) == 0:
            if log: print(self.players[self.player_move].name + " wins!")
            return True
        return False

    def debugMove(self):
        if (self.previous_declaration is not None) and (self.true_card[0] < self.previous_declaration[0]) and \
                    len(self.players[self.player_move].cards) == 1:
            print("[ERROR] Last played card should be valid (it is revealed, you cannot cheat)!")
            return False
        if np.array(self.true_card).size != 2:
            print("[ERROR] You put too many cards!")
            return False
        if self.true_card not in self.player_cards[self.player_move]:
            print("[ERROR] You do not have this card!")
            return False
        if self.true_card not in self.deck:
            print("[ERROR] There is no such card!")
            return False
        if (self.previous_declaration is not None) and len(self.pile) == 0:
            print("[ERROR] Inconsistency")
            return False
        if (self.previous_declaration is not None) and (self.declared_card[0] < self.previous_declaration[0]):
            print(len(self.pile))
            print(self.previous_declaration)
            print(self.declared_card)
            print(self.pile[-1])
            print("[ERROR] Improper move!")
            return False
        return True

    def debugGeneral(self):
        A = set(self.players[0].cards)
        B = set(self.players[1].cards)
        C = set(self.player_cards[0])
        D = set(self.player_cards[1])
        P = set(self.pile)
        E = set(self.game_deck)

        if not A == C:
            print("Error 001")
            return False
        if not B == D:
            print("Error 002")
            return False
        if not A | B | P == E:
            print("Error 003")
            print(A)
            print(B)
            print(P)
            print(E)
            return False
        return True

Analyze few moves...

In [ ]:
player1 = RandomPlayer("Player A")
player2 = RandomPlayer("Player B")
game = Game([player1, player2])

for i in range(6):
    game.takeTurn()

Player (1): Player A received:
[(11, 1), (12, 1), (9, 2), (13, 0), (14, 3), (12, 2), (14, 2), (9, 1)]
Player (2): Player B received:
[(9, 0), (9, 3), (11, 0), (14, 1), (12, 0), (13, 3), (10, 3), (13, 2)]


==== CURRENT STATE ================================
==== Player A MOVES ====
Player (0): Player A hand:
[(11, 1), (12, 1), (9, 2), (13, 0), (14, 3), (12, 2), (14, 2), (9, 1)]
Player (1): Player B hand:
[(9, 0), (9, 3), (11, 0), (14, 1), (12, 0), (13, 3), (10, 3), (13, 2)]
Pile: 
[]
Declared top card:
None

[+] Player A puts (14, 3) and declares (14, 3)
[!] Player B: I want to check
	You are wrong!
Feedback = Player A : checked this turn = True; I checked = False; I drew cards = False; revealed card = None; number of taken cards = 1
Feedback = Player B : checked this turn = True; I checked = True; I drew cards = True; revealed card = (14, 3); number of taken cards = 1
Cards taken: 
[(14, 3)]


==== CURRENT STATE ================================
==== Player B MOVES ====
Player (0): Pla

In [ ]:
### Some debug players
class DrawPlayer(Player):

    ### player's random strategy
    def putCard(self, declared_card):
        return "draw"

    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([False, False])

class SimplePlayer(Player):

    ### player's simple strategy
    def putCard(self, declared_card):

        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration

    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return np.random.choice([True, False], p=[0.3, 0.7])

In [ ]:
class ChatPlayer(Player):

    def __init__(self, name):
        super().__init__(name)
        self.seen_cards = [[0 for _ in range(4)] for _ in range(6)]
        self.seen_cards

    def putCard(self, declared_card):
        # Mark known cards
        for val, suit in self.cards:
            self.seen_cards[val - 9][suit] = 1

        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        # If it's the first card or pile is empty (declared_card is None), play the lowest card
        if declared_card is None:
            card = min(self.cards, key=lambda x: x[0])  # Play the smallest card
            declaration = card  # Declare the card honestly for now
        else:
            # If there is a declared card, either play honestly or bluff
            valid_cards = [card for card in self.cards if card[0] >= declared_card[0]]

            if valid_cards:
                # Choose a valid card and possibly bluff by declaring a higher card
                card = min(valid_cards, key=lambda x: x[0])  # Play the smallest valid card
                declaration = (card[0], card[1])  # Don't Bluff about the card value
            else:
                # No valid card, have to bluff with the smallest card
                card = min(self.cards, key=lambda x: x[0])  # Play the smallest card
                declaration = (declared_card[0], (declared_card[1] + 1) % 4)  # Bluff about the card value

        return card, declaration

    def checkCard(self, opponent_declaration):
        # Randomized decision to check or not, with more bias towards checking later in the game
        check_probability = 0.2 + (0.5 if len(self.cards) > 10 else 0)
        return np.random.choice([True, False], p=[check_probability, 1 - check_probability])

In [ ]:
import numpy as np
#from player import Player

class Nowak_Roszkiewicz(Player):

    def do_I_know_all_cards(self):
        # Sprawdź czy przypadkiem nie znamy już wszystkich kart
        if not self.I_know_all_cards:
            for value, suit in self.cards:
                self.cards_in_game[value - 9][suit] = 1
            if sum(sum(row) for row in self.cards_in_game) == 16:
                self.I_know_all_cards = True

    def __init__(self, name):
        super().__init__(name)
        self.firstMove = True
        self.num_opponent_cards = 8
        self.cards_in_game = [[0 for _ in range(4)] for _ in range(6)]
        # 0: Nie wiemy czy jest w grze
        # 1: Na pewno jest w grze
        self.I_know_all_cards = False
        self.turn = 0
        self.table = []

    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        super().getCheckFeedback(checked, iChecked, iDrewCards, revealedCard, noTakenCards, log)
        if(checked == False):
            return
        # Zapamiętaj że taka już była użyta
        if revealedCard is not None:
            value, suit = revealedCard
            self.cards_in_game[value - 9][suit] = 1
            self.do_I_know_all_cards()
        # Nieważne kto dobiera, zawsze karty znikają ze stołu
        for _ in range(noTakenCards):
            if self.table:
                self.table.pop()

    def putCard(self, declared_card):
        self.turn = self.turn + 1
        self.num_opponent_cards = 16 - len(self.cards) - len(self.table)

        # Dobierz jeśli nie możesz wygrać
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            self.table = self.table[:-3] if len(self.table) >= 3 else []
            return "draw"

        # Jeśli stół jest pusty, zawsze połóż najmniejszą
        if declared_card is None:
            card = min(self.cards, key=lambda x: x[0])
            declaration = card
            self.table.append((True, card))
            return card, declaration

        smallest_card = min(self.cards, key=lambda x: x[0])

        # Policzmy utility dla każdej możliwej karty, którą mam
        card_utilities = [0 for _ in range(len(self.cards))]
        for i, card in enumerate(self.cards):
            # Dokładanie najniższych zawsze będzie opłacalne
            card_utilities[i] += (smallest_card[0] - card[0]) # Dokładanie asów powinno mieć ujemną utility dlatego 13
            # Dołożenie karty poniżej progu powinno być nagradzane
            if declared_card is not None and card[0] < declared_card[0]:
                card_utilities[i] += declared_card[0] - card[0]
                # Ale mogą nas sprawdzić więc jakaś kara za blef
                # Kara powinna być zależna od tego jaką kartę dobieramy, a jesteśmy pewni tylko swoich kart
                i_would_draw = self.table[-2:] if len(self.table) >= 2 else self.table[:]
                utility_after_check = 0
                for draw_card in i_would_draw:
                    if(draw_card[0] == True): # To moja karta, więc wiem co to jest
                        if(draw_card[1][0] <= declared_card[0] - 2): # Jeśli moja karta jest co najmniej 2 mniejsza niż declared to nie chcę jej mieć
                            utility_after_check += (draw_card[1][0] - declared_card[0])
                        # W innym przypadku możemy dobrać o 1 mniejszą załóżmy bez kary
                    else:
                        # Za każdą kartę przeciwnika stała kara 1
                        utility_after_check -= 1
                card_utilities[i] += utility_after_check

        max_utility = max(card_utilities)
        best_indexes = [i for i, u in enumerate(card_utilities) if u == max_utility]
        best_card = self.cards[best_indexes[0]]

        declaration = None

        if(best_card[0] >= declared_card[0]):
            declaration = best_card
        else:
            sorted_cards = sorted(self.cards, key=lambda x: x[0])
            for declare in sorted_cards:
                if declare[0] >= declared_card[0]:
                    declaration = declare
                    break

        if(declaration == None):
            self.table = self.table[:-3] if len(self.table) >= 3 else []
            return "draw"

        self.table.append((True, best_card))
        return best_card, declaration

    def checkCard(self, opponent_declaration):
        if opponent_declaration is not None:
            self.table.append((False, opponent_declaration))
        self.num_opponent_cards = 16 - len(self.cards) - len(self.table)

        # Sprawdzam jeśli powiedział moją
        if opponent_declaration in self.cards:
            return True
        # Jeśli znam wszystkie karty, a on powiedział inną to sprawdzam
        if self.I_know_all_cards and self.cards_in_game[opponent_declaration[0] - 9][opponent_declaration[1]] == 0:
            return True
        # Jeśli mam tylko 1, a i tak nie będę mógł jej dołożyć w następnym ruchu to sprawdzaj, bo inaczej i tak bym musiał dobrać
        if len(self.cards) == 1 and opponent_declaration is not None and self.cards[0][0] < opponent_declaration[0]:
            return True
        # Jeśli mam mniej niż przeciwnik, to sprawdzam
        if self.num_opponent_cards == 1 and len(self.cards) - self.num_opponent_cards >= 1:
            return True
        if self.num_opponent_cards <= 3 and len(self.cards) - self.num_opponent_cards >= 2:
            return np.random.choice([True, False], p=[0.75, 0.25])
        if self.num_opponent_cards <= 5 and len(self.cards) - self.num_opponent_cards >= 3:
            return np.random.choice([True, False], p=[0.5, 0.5])
        if len(self.cards) - self.num_opponent_cards >= 4:
            return np.random.choice([True, False], p=[0.25, 0.75])

        return False

In [ ]:
class BetterWorsePlayer(Player):

    def do_I_know_all_cards(self):
        # Sprawdź czy przypadkiem nie znamy już wszystkich kart
        if not self.I_know_all_cards:
            for value, suit in self.cards:
                self.cards_in_game[value - 9][suit] = 1
            if sum(sum(row) for row in self.cards_in_game) == 16:
                self.I_know_all_cards = True

    def __init__(self, name):
        super().__init__(name)
        self.firstMove = True
        self.num_opponent_cards = 8
        self.cards_in_game = [[0 for _ in range(4)] for _ in range(6)]
        # 0: Nie wiemy czy jest w grze
        # 1: Na pewno jest w grze
        self.I_know_all_cards = False
        self.table = []

    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        super().getCheckFeedback(checked, iChecked, iDrewCards, revealedCard, noTakenCards, log)
        if(checked == False):
            return
        # Zapamiętaj że taka już była użyta
        if revealedCard is not None:
            value, suit = revealedCard
            self.cards_in_game[value - 9][suit] = 1
            self.do_I_know_all_cards()
        # Nieważne kto dobiera, zawsze karty znikają ze stołu
        for _ in range(noTakenCards):
            if self.table:
                self.table.pop()


    def putCard(self, declared_card):
        self.num_opponent_cards = 16 - len(self.cards) - len(self.table)

        # Dobierz jeśli nie możesz wygrać
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        # Jeśli stół jest pusty, zawsze połóż najmniejszą
        if declared_card is None:
            card = min(self.cards, key=lambda x: x[0])
            declaration = card
            self.table.append((True, card))
            return card, declaration

        # Jeszcze dobrą (chyba) startegią jest kładzenie wysoka-niska-wysoka, bo jeśli nas sprawdzi to dostaniemy zawsze przynajmniej 1 wysoką
        better = len([card for card in self.cards if card[0] >= declared_card[0]])
        worse = len([card for card in self.cards if card[0] < declared_card[0]])

        sorted_cards = sorted(self.cards, key=lambda x: x[0])

        declaration = None
        if better > worse:
            for card in sorted_cards:
                if card[0] >= declared_card[0]:
                    best_card = card
                    declaration = card
                    break
        else:
            best_card = sorted_cards[0]
            for declare in sorted_cards:
                if declare[0] >= declared_card[0]:
                    declaration = declare
                    break
        if not declaration:
            declaration = (14, 0)

        self.table.append((True, best_card))
        return best_card, declaration

    def checkCard(self, opponent_declaration):
        if opponent_declaration is not None:
            self.table.append((False, opponent_declaration))
        self.num_opponent_cards = 16 - len(self.cards) - len(self.table)

        # Sprawdzam jeśli powiedział moją
        if opponent_declaration in self.cards:
            return True
        # Jeśli znam wszystkie karty, a on powiedział inną to sprawdzam
        if self.I_know_all_cards and self.cards_in_game[opponent_declaration[0] - 9][opponent_declaration[1]] == 0:
            return True
        # Jeśli mam tylko 1, a i tak nie będę mógł jej dołożyć w następnym ruchu to sprawdzaj, bo inaczej i tak bym musiał dobrać
        if len(self.cards) == 1 and opponent_declaration is not None and self.cards[0][0] < opponent_declaration[0]:
            return True
        # Jeśli mam mniej niż przeciwnik, to sprawdzam
        if self.num_opponent_cards == 1 and len(self.cards) - self.num_opponent_cards >= 1:
            return True
        if self.num_opponent_cards <= 3 and len(self.cards) - self.num_opponent_cards >= 2:
            return np.random.choice([True, False], p=[0.9, 0.1])
        if self.num_opponent_cards <= 5 and len(self.cards) - self.num_opponent_cards >= 3:
            return np.random.choice([True, False], p=[0.7, 0.3])
        if len(self.cards) - self.num_opponent_cards >= 4:
            return np.random.choice([True, False], p=[0.6, 0.4])

        return False

In [ ]:
### Perform a full game 100 times
stats_wins = [0, 0]
stats_moves = [0, 0]
stats_cheats = [0, 0]
stats_errors = [0, 0]
stats_cards = [0, 0]
stats_checks = [0, 0]
stats_draw_decisions = [0, 0]
stats_pile_size = 0

repeats = 100
errors = 0

for t in range(100):
    player1 = Nowak_Roszkiewicz("Nowak Roszkiewicz")
    player2 = BetterWorsePlayer("Better Worse")
    game = Game([player1, player2], log = False)

    error = False
    while True:
        valid, player = game.takeTurn(log = False)
        if not valid:
            error = True
            stats_errors[player] += 1
            errors += 1
            break
        if game.isFinished(log = False):
            stats_wins[player] += 1
            break

    stats_pile_size += len(game.pile)
    if not error:
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

stats_pile_size /= (repeats - errors)
for j in range(2):
    stats_moves[j] /= (repeats - errors)
    stats_cheats[j] /= (repeats - errors)
    stats_checks[j] /= (repeats - errors)
    stats_draw_decisions[j] /= (repeats - errors)
    stats_cards[j] /= (repeats - errors)

print("Better worse")
print("Wins:")
print(stats_wins)
print("Moves:")
print(stats_moves)
print("Cards:")
print(stats_cards)
print("Pile size:")
print(stats_pile_size)
print("Checks:")
print(stats_checks)
print("Draw decisions:")
print(stats_draw_decisions)
print("Cheats:")
print(stats_cheats)
print("Errors:")
print(stats_errors)
print("Total errors:")
print(errors)

Better worse
Wins:
[65, 35]
Moves:
[7.82, 7.67]
Cards:
[2.91, 5.44]
Pile size:
7.65
Checks:
[1.04, 1.45]
Draw decisions:
[0.12, 0.0]
Cheats:
[1.84, 1.25]
Errors:
[0, 0]
Total errors:
0


In [ ]:
### Perform a full game 100 times
stats_wins = [0, 0]
stats_moves = [0, 0]
stats_cheats = [0, 0]
stats_errors = [0, 0]
stats_cards = [0, 0]
stats_checks = [0, 0]
stats_draw_decisions = [0, 0]
stats_pile_size = 0

repeats = 100
errors = 0

for t in range(100):
    player1 = Nowak_Roszkiewicz("Nowak Roszkiewicz")
    player2 = SimplePlayer("Simple")
    game = Game([player1, player2], log = False)

    error = False
    while True:
        valid, player = game.takeTurn(log = False)
        if not valid:
            error = True
            stats_errors[player] += 1
            errors += 1
            break
        if game.isFinished(log = False):
            stats_wins[player] += 1
            break

    stats_pile_size += len(game.pile)
    if not error:
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

stats_pile_size /= (repeats - errors)
for j in range(2):
    stats_moves[j] /= (repeats - errors)
    stats_cheats[j] /= (repeats - errors)
    stats_checks[j] /= (repeats - errors)
    stats_draw_decisions[j] /= (repeats - errors)
    stats_cards[j] /= (repeats - errors)

print("Simple")
print("Wins:")
print(stats_wins)
print("Moves:")
print(stats_moves)
print("Cards:")
print(stats_cards)
print("Pile size:")
print(stats_pile_size)
print("Checks:")
print(stats_checks)
print("Draw decisions:")
print(stats_draw_decisions)
print("Cheats:")
print(stats_cheats)
print("Errors:")
print(stats_errors)
print("Total errors:")
print(errors)

Simple
Wins:
[70, 30]
Moves:
[9.27, 9.04]
Cards:
[3.77, 7.12]
Pile size:
5.11
Checks:
[2.48, 2.69]
Draw decisions:
[0.09, 0.0]
Cheats:
[1.78, 3.11]
Errors:
[0, 0]
Total errors:
0


In [ ]:
### Perform a full game 100 times
stats_wins = [0, 0]
stats_moves = [0, 0]
stats_cheats = [0, 0]
stats_errors = [0, 0]
stats_cards = [0, 0]
stats_checks = [0, 0]
stats_draw_decisions = [0, 0]
stats_pile_size = 0

repeats = 100
errors = 0

for t in range(100):
    player1 = Nowak_Roszkiewicz("Nowak Roszkiewicz")
    player2 = ChatPlayer("Chat")
    game = Game([player1, player2], log = False)

    error = False
    while True:
        valid, player = game.takeTurn(log = False)
        if not valid:
            error = True
            stats_errors[player] += 1
            errors += 1
            break
        if game.isFinished(log = False):
            stats_wins[player] += 1
            break

    stats_pile_size += len(game.pile)
    if not error:
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

stats_pile_size /= (repeats - errors)
for j in range(2):
    stats_moves[j] /= (repeats - errors)
    stats_cheats[j] /= (repeats - errors)
    stats_checks[j] /= (repeats - errors)
    stats_draw_decisions[j] /= (repeats - errors)
    stats_cards[j] /= (repeats - errors)

print("Simple")
print("Wins:")
print(stats_wins)
print("Moves:")
print(stats_moves)
print("Cards:")
print(stats_cards)
print("Pile size:")
print(stats_pile_size)
print("Checks:")
print(stats_checks)
print("Draw decisions:")
print(stats_draw_decisions)
print("Cheats:")
print(stats_cheats)
print("Errors:")
print(stats_errors)
print("Total errors:")
print(errors)

Simple
Wins:
[73, 27]
Moves:
[8.71, 8.51]
Cards:
[3.45, 4.18]
Pile size:
8.37
Checks:
[1.44, 1.87]
Draw decisions:
[0.03, 0.02]
Cheats:
[1.42, 0.53]
Errors:
[0, 0]
Total errors:
0
